In [1]:
import json
import re
import tqdm
from IPython.display import Markdown

## prepare raw wikiextract

In [2]:
%%bash
cat english.words| jq -s . > english.json

In [2]:
with open('english.json', 'r') as f:
    english_vocab_db = json.load(f)

## helper functions to filter senses, synonyms and antonyms

In [3]:
def filter_senses(senses_list, word):
    filtered_senses = []
    for entry in senses_list:
        if 'glosses' in entry:
            glosses = entry['glosses']
            for gloss_entry in glosses:
                for gloss in gloss_entry.split(';'):
                    s = re.sub('\s+', ' ', re.sub("[\(\[].*?[\)\]]", "", gloss).replace(".", "")).strip()
                    if len(s) > 0 and word not in s:
                        filtered_senses.append(s)
    return filtered_senses

In [4]:
def filter_synonyms(synonyms_list):
    filtered_synonyms = []
    for entry in synonyms_list:
        if 'word' in entry:
            filtered_synonyms.append(entry['word'])
    return filtered_synonyms

In [5]:
def filter_antonyms(antonym_list):
    filtered_antonyms = []
    for entry in antonym_list:
        if 'word' in entry:
            filtered_antonyms.append(entry['word'])
    return filtered_antonyms

In [6]:
def add_wiktionary_item(item, db):
    item = {}
    if 'word' not in item.keys():
        return
    

## build a dictionary_database from the raw extract
using words as keys, enumerate those keys (e.g. `house`, `house1`, `house2` etc.) on duplicates

In [7]:
dictionary_database = {}
for item in tqdm.tqdm(english_vocab_db):
    if 'word' in item:
        word = item['word']
        i = 0
        while word in dictionary_database:
            i += 1
            word = f"{item['word']}{i}"
        dictionary_database[word] = item
        

100%|██████████| 1023195/1023195 [00:00<00:00, 1386092.63it/s]


In [8]:
class NoDataException(Exception):
    pass

def get_attribute(word:str, attr:str, db = dictionary_database):
    if word not in db:
        raise NoDataException
    item = db[word]
    if attr not in item:
        raise NoDataException
    return item[attr]

def get_senses(word: str):
    
    senses = get_attribute(word, 'senses')
    
    return filter_senses(senses, word)

def get_synonyms(word: str):
    syns = get_attribute(word, 'synonyms')
    
    return filter_synonyms(syns)

def get_antonyms(word: str):
    ants = get_attribute(word, 'antonyms')
    
    return filter_antonyms(ants)

def display_info(word: str):
    
    if word not in dictionary_database:
        raise NoDataException
    
    try:
        senses = get_senses(word)
    except NoDataException:
        senses = []
    
    try:
        synonyms = get_synonyms(word)
    except NoDataException:
        synonyms = []
    
    try:
        antonyms = get_antonyms(word)
    except NoDataException:
        antonyms = []
        
    senses_str = '\n'.join([f"* {sense}" for sense in senses])
    
    synonyms_str = '\n'.join([f"* {syn}" for syn in synonyms])
    
    antonyms_str = '\n'.join([f"* {ant}" for ant in antonyms])
    
    markdown_string = f"""
## {word}

### Senses:

{senses_str}

### Synonyms:

{synonyms_str}

### Antonyms:

{antonyms_str}
    
    """
    
    return Markdown(markdown_string)

In [9]:
import random
list_words = []
for word in tqdm.tqdm(dictionary_database.keys()):
    item = dictionary_database[word]
    try:
        senses = get_senses(word)
        if len(senses) > 1:
            list_words.append(word)
            
    except NoDataException:
        pass
    

100%|██████████| 1023195/1023195 [00:06<00:00, 159434.15it/s]


In [18]:
display(display_info("cockmonger"))


## cockmonger

### Senses:

* A frequent consumer of cocks
* A procurer of cocks
* A person with a large penis

### Synonyms:



### Antonyms:


    
    

----
## algorithm to build a grid of words

In [358]:
import numpy as np
import random
from string import digits

class WordInfo(object):
    def __init__(self, word:str, y:int, x:int, is_vertical: bool, database=dictionary_database):
        self._dictionary_database = dictionary_database
        self._y = y
        self._x = x
        self._word = word
        self._hint = None
        self._is_vertical = is_vertical
        
        self.choose_info()
    
    def get_best_antonym(self) -> str:
        antonyms = get_antonyms(self._word)
        return random.choice(antonyms)
    
    def get_best_synonym(self) -> str:
        synonyms = get_synonyms(self._word)
        return random.choice(synonyms)
    
    def get_best_sense(self) -> str:
        senses = get_senses(self._word)
        return random.choice(senses)
        
    def choose_info(self):
        # first choose antonyms, then synonyms, then senses
        
        try:
            self._hint = f"opposite of {self.get_best_antonym()}"
        except NoDataException:
            pass
        
        try:
            self._hint = f"other word for {self.get_best_synonym()}"
        except NoDataException:
            pass
        
        self._hint = self.get_best_sense()
    
    def get_hint(self) -> str:
        return self._hint
    
    def get_hint_location(self):
        x = self._x if self._is_vertical else self._x - 1
        y = self._y if self._is_vertical else self._y - 1
        return (y, x)
    
    def is_vertical(self):
        return self._is_vertical

def create_word_grid(w: int, h: int, word_list = list_words, target_density = 0.5):
    grid = np.full(shape=(h,w), dtype=np.unicode, fill_value = ' ')
    
    locations = {}
    
    word_hints = {}
    
    def store_location(char: str, y: int, x: int):
        assert len(char) == 1
        
        if char not in locations:
            locations[char] = []
        
        locations[char].append([y,x])
        
    remove_digits = str.maketrans('', '', digits)
    n_words = len(list_words)
    
    def get_word(max_length: int, min_length = 0):
        assert max_length > 1
        
        index = random.randint(0,n_words-1)
        word = list_words[index][:]
        
        while len(word) >= max_length or not word.isalnum() or len(word) <= min_length:
            index = random.randint(0,n_words-1)
            word = list_words[index][:]
        
        return word
            
    def normalize_word(word:str):
        word = word.translate(remove_digits)
        return word.lower()
    
    def place_word(word:str, y: int, x:int, vertical:bool = False):
        normalized_word = normalize_word(word)
        n = len(normalized_word)
        if vertical:
            assert grid.shape[0] - n >= y
            for i, char in enumerate(normalized_word):
                grid[y + i,x] = char
                store_location(char, y+i, x)
        else:
            assert grid.shape[1] - n >= x
            for i, char in enumerate(normalized_word):
                grid[y,x + i] = char
                store_location(char, y, x+i)
        
        word_hints[normalized_word] = WordInfo(word, y, x, vertical)
        
        
    
    def density():
        return 1 - (grid == " ").sum() / (w * h)
    
    
    
    def check_if_fits(word:str, y:int, x:int, vertical:bool):
        n = len(word)
        if vertical:
            
            # check if there is space before and after
            if y - 1 >= 0 and grid[y - 1, x] != " ":
                return False
            if y + n < grid.shape[0] - 1 and grid[y+n,x] != " ":
                return False
            
            if grid.shape[0] - n < y or y < 0:
                #print("over board")
                return False
            
            for i, char in enumerate(word):
                char_x = x
                char_y = y + i
                
                if not (grid[char_y, char_x] == " " or grid[char_y, char_x] == char):
                    #print("not matching")
                    return False
                
                if grid[char_y, char_x] == " ":
                    # check for horizonatal neighbors:
                    if char_x - 1 >= 0 and grid[char_y, char_x - 1] != " ":
                        #print("3")
                        return False
                    if char_x + 1 < grid.shape[1] and grid[char_y, char_x + 1] != " ":
                        #print("4")
                        return False
        
        else:
            
            # check if there is space before and after
            if x - 1 >= 0 and grid[y, x - 1] != " ":
                return False
            if x + n < grid.shape[1] - 1 and grid[y,x + n] != " ":
                return False
            
            if grid.shape[1] - n < x or x < 0:
                #print("over board")
                return False
            
            for i, char in enumerate(word):
                char_x = x + i
                char_y = y
                
                if not (grid[char_y, char_x] == " " or grid[char_y, char_x] == char):
                    #print("not matching")
                    return False
                
                if grid[char_y, char_x] == " ":
                    # check for vertical neighbors:
                    if char_y - 1 >= 0 and grid[char_y - 1, char_x] != " ":
                        #print("1")
                        return False
                    if char_y + 1 < grid.shape[0] and grid[char_y + 1, char_x] != " ":
                        #print("2")
                        return False
        
        return True
        
    
    def get_crossover(word: str):
        # returns Tuple of: (y,x, is_vertical?) or None
        
        shuffled_order = list(range(len(word)))
        random.shuffle(shuffled_order)
        
        for index in shuffled_order:
            # check for existing locations
            char = word[index]
            if char in locations:
                char_locations = locations[char]
                
                for char_loc in char_locations:
                    # test vertical
                    y = char_loc[0] - index
                    x = char_loc[1]
                    
                    if check_if_fits(word, y, x, vertical=True):
                        return (y,x,True)
                    
                    # test horizontal
                    y = char_loc[0]
                    x = char_loc[1] - index
                    
                    if check_if_fits(word, y, x, vertical=False):
                        return (y,x,False)
        
        return None
    
    min_shape = min(w,h,30)
    
    # place first word:
    first_word = get_word(max_length=min_shape, min_length=min(10,grid.shape[1] - 2))
    
    # find random place:
    x = random.randint(0, grid.shape[1] - len(first_word) - 1)
    y = random.randint(0, grid.shape[0] - 1)
    
    place_word(first_word, y, x, vertical=False)
                
    i = 0
    
    
    current_density = density()
    
    while current_density < target_density:
        word = get_word(max_length=(1 - current_density ** 0.4) * min_shape,
                        min_length=max(min(10, 0.5 * (1 - current_density ** 0.3) * min_shape), 2))
        
        normalized_word = normalize_word(word)
        
        if normalized_word in word_hints:
            continue
        
        # check if matching characters exist:
        crossover = get_crossover(normalized_word)
        
        i += 1
        if i % 100000 == 0:
            print(i)
        if i > 100000:
            break
        
        if crossover == None:
            current_density = density()
            continue
        
        y,x,is_vertical = crossover
        
        place_word(word, y,x, is_vertical)
        
        current_density = density()
    
    print(i)
    return grid, word_hints
        
        
        
        
        
    

In [359]:
%%prun
gr, infos = create_word_grid(16*3,9*3, target_density=0.55)
def grid2mdown(grid):
    h,w  = grid.shape
    
    md_str = "     ".join(["|"] * (w+1)) + "\n"
    md_str += ":---:".join(["|"] * (w+1)) + "\n"
    
    for y in range(h):
        md_str += "|" + "|".join([(f"**{grid[y,x]}**" if grid[y,x] != " " else "     ") for x in range(w)]) + "|\n"
    
    return Markdown(md_str)

display(grid2mdown(gr))
    
    

968


|     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |     |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|     |     |     |     |**c**|     |**a**|**e**|**r**|**o**|**p**|**h**|**o**|**b**|**i**|**c**|     |**m**|**g**|**m**|     |**f**|**a**|**d**|**d**|**y**|     |**f**|     |     |**o**|     |**c**|     |     |**g**|**r**|**a**|**y**|     |     |     |     |**q**|**u**|**a**|**y**|     |
|**f**|**b**|**i**|     |**l**|     |     |     |**h**|     |     |**i**|     |**f**|     |     |     |**m**|     |     |**r**|     |**c**|     |**a**|     |     |**o**|**v**|**e**|**r**|**w**|**o**|**r**|**d**|     |**p**|     |     |**t**|     |**s**|     |**u**|     |     |     |**r**|
|**i**|     |     |**s**|**m**|**s**|     |     |**i**|     |     |**a**|     |     |**m**|     |     |**p**|**a**|**c**|**i**|**f**|**i**|**s**|**m**|     |     |**o**|     |**x**|     |     |**m**|     |     |**p**|**a**|**i**|**g**|**e**|     |**i**|     |**a**|     |**h**|     |**e**|
|**g**|**l**|**e**|**e**|     |**y**|     |**c**|**a**|**r**|**o**|**t**|**e**|**n**|**o**|**l**|     |     |     |**o**|     |     |**c**|     |     |**a**|**u**|**f**|     |**p**|**o**|**p**|**p**|**l**|**e**|     |     |**m**|     |**n**|     |**l**|     |**d**|**r**|**o**|**f**|**f**|
|**u**|     |     |**c**|     |**m**|     |     |**n**|     |     |**t**|     |     |**r**|     |     |**d**|**o**|**g**|**g**|**i**|**l**|**y**|     |     |     |     |**s**|     |     |     |**r**|     |     |**g**|**y**|**p**|     |**t**|     |**v**|     |**r**|     |**r**|     |**a**|
|**r**|     |     |**o**|**r**|**p**|**h**|**a**|**n**|**r**|**y**|     |     |     |**t**|     |**e**|     |     |**n**|     |     |**e**|     |     |     |**s**|**p**|**o**|**n**|**g**|**i**|**o**|**s**|**e**|     |     |**r**|     |     |     |**e**|     |**u**|     |**s**|     |**n**|
|**a**|     |     |**n**|     |**t**|     |     |**o**|     |     |     |**r**|     |**a**|**f**|**r**|**o**|**d**|**i**|**t**|**e**|     |**a**|     |**a**|     |     |**l**|     |     |     |**m**|     |     |     |**c**|**o**|**l**|     |**d**|**r**|**i**|**p**|     |**e**|     |     |
|**l**|     |**e**|**d**|**d**|**o**|     |     |**n**|**g**|**p**|     |**y**|     |**l**|     |**v**|     |     |**z**|     |**v**|     |**m**|     |**f**|     |     |**a**|**f**|**v**|     |**i**|     |     |     |     |**p**|     |**e**|     |**i**|     |**l**|     |**l**|     |     |
|     |**e**|     |**e**|     |**m**|     |**s**|     |     |     |     |**n**|     |     |**n**|     |     |**w**|**a**|**g**|**e**|**n**|**b**|**o**|**o**|**m**|     |**r**|     |**a**|**r**|**s**|**e**|**w**|**i**|**p**|**e**|     |**m**|**i**|**n**|**d**|**e**|**l**|**o**|     |**u**|
|**u**|**l**|**l**|**r**|     |**a**|**n**|**t**|**e**|**c**|**r**|**i**|**t**|**i**|**c**|**a**|**l**|     |     |**n**|     |**n**|     |**i**|     |**u**|     |     |**y**|     |**t**|     |**e**|     |     |**m**|     |**r**|**o**|**b**|     |**g**|     |     |     |**r**|     |**n**|
|     |**e**|     |     |     |**t**|     |**e**|     |**r**|     |     |     |     |     |**n**|     |**p**|     |**c**|**u**|**e**|     |**t**|     |**l**|**a**|**t**|     |**m**|     |     |     |     |     |**m**|     |     |     |**r**|     |     |**t**|     |**l**|**e**|**s**|**s**|
|     |**c**|**u**|**r**|**r**|**i**|**e**|**r**|     |**a**|     |**o**|**x**|**a**|**z**|**o**|**l**|**i**|**n**|**e**|     |     |     |     |**w**|     |     |     |**s**|**e**|**i**|**s**|**m**|**o**|**l**|**o**|**g**|**i**|**c**|**a**|**l**|**l**|**y**|     |     |     |     |**t**|
|     |**t**|     |     |     |**c**|     |**c**|     |**c**|     |     |     |     |     |**m**|     |**l**|     |     |**m**|     |**s**|     |**o**|     |     |     |**h**|     |     |**c**|     |     |     |**r**|     |     |     |**i**|     |     |**p**|     |     |     |     |**i**|
|**c**|**r**|**a**|**m**|**p**|     |**m**|**o**|**c**|**k**|**e**|**r**|**y**|     |**a**|**a**|**a**|**e**|     |**t**|**o**|**s**|**h**|**e**|**r**|     |**r**|**h**|**a**|**p**|**s**|**o**|**d**|**i**|**s**|**t**|**i**|**c**|     |**d**|     |**p**|**a**|**p**|**p**|**y**|     |**t**|
|     |**e**|     |     |     |     |     |**r**|     |**i**|     |     |     |**d**|     |**c**|     |     |     |     |**g**|     |**e**|     |**m**|     |**f**|     |     |**e**|     |     |     |     |     |**a**|     |**d**|     |     |**s**|     |**l**|     |**r**|     |**o**|**c**|
|     |     |**d**|**m**|     |**c**|**r**|**y**|**i**|**n**|**g**|     |**p**|**u**|**s**|**h**|**b**|**a**|**l**|**l**|     |**u**|**n**|**p**|**h**|**o**|**t**|**o**|**g**|**r**|**a**|**p**|**h**|**a**|**b**|**l**|**e**|     |**m**|     |**h**|     |     |     |**i**|     |     |**h**|
|     |**g**|     |**u**|     |**m**|     |     |     |**g**|     |     |     |**e**|     |**i**|     |**a**|     |     |**m**|     |**a**|     |**o**|     |     |     |     |     |**a**|     |     |**l**|     |**i**|     |     |**u**|     |**e**|**s**|**k**|**i**|**m**|**o**|     |     |
|     |**c**|**o**|**l**|**e**|**a**|**d**|     |     |     |     |**p**|     |     |     |**n**|     |**b**|     |     |**i**|     |**n**|     |**l**|     |**u**|**n**|**v**|**i**|**s**|**u**|**a**|**l**|**i**|**z**|**a**|**b**|**l**|**e**|     |**i**|     |     |**i**|     |     |     |
|     |**e**|     |**t**|     |     |     |     |     |**s**|     |**l**|**e**|**t**|**t**|**e**|**r**|     |**p**|**u**|**n**|**t**|**i**|**t**|**e**|     |     |     |     |     |     |     |**d**|     |     |**e**|     |     |**t**|     |     |**m**|     |     |**g**|     |     |     |
|     |     |**l**|**i**|**g**|**h**|**t**|**s**|**c**|**a**|**p**|**e**|     |     |     |**r**|     |     |     |     |     |     |**g**|     |     |     |     |**c**|**t**|**c**|     |     |**s**|**i**|**e**|     |     |**l**|**a**|**m**|**b**|**a**|**s**|**t**|**e**|**d**|     |     |
|     |     |     |**t**|     |     |**b**|     |     |**d**|     |**a**|     |     |**p**|**y**|**r**|**a**|**z**|**o**|**l**|**e**|**a**|**m**|**i**|**d**|**e**|     |**o**|     |**f**|     |     |     |     |     |**l**|     |**n**|     |     |     |     |     |**n**|     |     |**g**|
|     |     |     |**h**|     |**m**|     |**b**|     |**d**|     |**d**|     |     |**a**|     |     |     |**e**|     |     |     |**n**|     |     |**r**|     |     |**i**|     |**a**|**z**|**a**|**c**|**y**|**t**|**o**|**s**|**i**|**n**|**e**|     |     |**d**|**o**|**t**|**t**|**y**|
|     |     |**s**|**e**|**r**|**i**|**c**|**a**|**t**|**e**|     |**i**|     |     |**l**|     |     |     |**r**|     |**n**|     |**r**|     |     |**u**|     |     |**n**|     |**w**|     |     |     |     |     |**u**|     |**m**|     |     |     |     |     |**u**|     |     |**m**|
|     |**j**|     |**i**|     |**n**|     |**n**|     |**n**|     |**n**|     |**d**|**e**|**p**|**o**|**s**|**i**|**t**|**a**|**r**|**y**|     |     |**m**|**o**|**r**|**g**|**e**|**n**|     |**c**|**a**|**i**|**r**|**n**|**g**|**o**|**r**|**m**|**s**|     |     |**s**|**k**|**i**|**n**|
|     |**o**|     |**s**|     |**y**|     |**d**|     |     |     |**g**|     |     |     |     |**u**|     |**b**|     |**i**|     |     |**r**|     |**b**|     |**a**|     |**d**|     |**v**|     |**m**|     |     |     |     |**u**|     |     |**u**|     |**r**|     |**o**|     |**i**|
|     |**h**|**a**|**m**|**f**|**a**|**t**|     |**c**|**y**|**c**|**l**|**o**|**p**|**a**|**e**|**d**|**i**|**a**|     |**r**|     |**c**|**o**|**l**|**l**|**a**|**r**|     |**e**|     |**h**|**e**|**a**|**d**|**h**|**o**|**u**|**s**|**e**|     |**r**|**c**|**s**|     |**k**|     |**c**|
|     |**n**|     |     |     |**n**|     |     |     |     |     |**y**|     |     |     |     |     |**t**|     |**t**|**a**|**u**|     |**e**|     |**e**|     |**e**|     |**n**|     |**d**|     |**r**|     |     |     |     |     |     |     |**e**|     |**o**|     |**o**|     |     |


         854104 function calls (854099 primitive calls) in 0.447 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   385008    0.289    0.000    0.306    0.000 <ipython-input-358-bafd40978d5d>:111(check_if_fits)
      968    0.072    0.000    0.383    0.000 <ipython-input-358-bafd40978d5d>:174(get_crossover)
   395346    0.018    0.000    0.018    0.000 {built-in method builtins.len}
      969    0.017    0.000    0.027    0.000 <ipython-input-358-bafd40978d5d>:106(density)
      969    0.008    0.000    0.008    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      973    0.007    0.000    0.016    0.000 <ipython-input-358-bafd40978d5d>:72(get_word)
        1    0.005    0.005    0.444    0.444 <ipython-input-358-bafd40978d5d>:54(create_word_grid)
     9573    0.005    0.000    0.007    0.000 random.py:238(_randbelow_with_getrandbits)
     5549    0.003    0.000    0.007    0.000 random.py:291(randrange)
      968    0.002

In [279]:
i=220

word = list(infos.keys())[i]

print(word, "--", infos[word].get_hint())

edie -- surname


In [360]:
display(display_info("wagenboom"))


## wagenboom

### Senses:

* A South African proteaceous tree
* The tough wood of this tree, used for making wagon wheels

### Synonyms:

* waboom
* wagon tree

### Antonyms:


    
    

In [374]:
from enum import Enum

class HintOrientation(Enum):
    VERTICAL = 0
    HORIZONTAL = 1
    BOTH = 2
    
class FieldType(Enum):
    EMPTY = 0
    HINT = 1
    LETTER = 2

class Field(object):
    def __init__(self, field_type: FieldType = FieldType.EMPTY):
        self._field_type = field_type
    
    def get_type(self) -> FieldType:
        return self._field_type
    
    def get_content(self) -> str:
        return None

class HintField(Field):
    def __init__(self, horizontal_hint: str = None, vertical_hint: str = None):
        super().__init__(field_type=FieldType.HINT)
        
        self._horizontal_hint = horizontal_hint
        self._vertical_hint = vertical_hint
    
    def get_horizontal_hint(self) -> str:
        return self._horizontal_hint
    
    def get_vertical_hint(self) -> str:
        return self._vertical_hint
    
    def set_horizintal_hint(hint: str):
        self._horizontal_hint = hint
    
    def set_vertical_hint(hint:str):
        self._vertical_hint = hint

class LetterField(Field):
    def __init__(self, letter: str):
        assert len(letter) <= 1
        
        super().__init__(field_type = FieldType.LETTER)
        
        self._letter = letter
        self._revealed = False
    
    def get_content(self) -> str:
        return self._letter
    
    def reveal(self):
        self._revealed = True
    
    def is_revealed(self) -> bool:
        return self._revealed

class Grid(object):
    def __init__(self, width: int, height: int, density = 0.5):
        self._width = width
        self._height = height
        self._density = density
        self._grid = []
        self._build_grid()
    
    def _build_grid(self):
        raw_grid, word_infos = create_word_grid(self._width - 1, self._height - 1, target_density=self._density)
        
        # note: we will append an additional row and column, to have enough space to place hint fields
        
        self._grid = [[Field()] * self._width] # initialize with empty row
        
        for y in range(self._height):
            row = [Field()] # initialize row with empty column
            for x in range(self._width):
                raw_cell = raw_grid[y-1,x-1]
                if raw_cell == " ":
                    row.append(Field())
                else:
                    row.append(LetterField(raw_cell))
            
            self._grid.append(row)
        
        # place hint fields:
        for word, info in word_infos.items():
            y,x = info.get_hint_location()
            # correct offset
            y += 1
            x += 1
            
            cell = self._grid[y][x]
            
            # check if we already have a hint here:
            if cell.get_type() == FieldType.HINT:
                if info.is_vertical():
                    cell.set_vertical_hint(info.get_hint())
                else:
                    cell.set_horizintal_hint(info.get_hint())
            elif cell.get_type() == FieldType.LETTER:
                # edge case: a word has "eaten up" another one, skipping that case
                pass
            
            else:
                if info.is_vertical():
                    self._grid[y][x] = HintField(vertical_hint=info.get_hint())
                else:
                    self._grid[y][x] = HintField(horizontal_hint=info.get_hint())

                        
class Crossword(object):
    def __init__(self, width: int, height: int, database: dict):
        self._width = width
        self._height = height
        self._database = database
        self._grid = Grid(width, height)
    
    
        
        

In [375]:
cw = Crossword(16*3, 9*3, english_vocab_db)

234
